In [ ]:
import smtplib
import email, ssl


from string import Template
from email.mime.base import MIMEBase
from email import encoders


from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

MY_ADDRESS = 'YourEmail'         #Write your email here
PASSWORD = 'YourPassword'        #Write the password here

def getnumber(i):                         #you can use any kind of algorithm to generate the token number for each mail.
    return "TA - "+ str(138+i)

def get_contacts(filename):
    """
    Return two lists names, emails containing names and email addresses
    read from a file specified by filename.
    """
    
    names = []
    names2 = []
    emails = []
    with open(filename, mode='r', encoding='utf-8') as contacts_file:
        for a_contact in contacts_file:
            names.append(a_contact.split(',')[2])
            names2.append(a_contact.split(',')[3])
            emails.append(a_contact.split(',')[1])
    return names, names2, emails

def read_template(filename):
    """
    Returns a Template object comprising the contents of the 
    file specified by filename.
    """
    
    return Template("\nDear ${PERSON_NAME} & ${PERSON_NAME2},<br>\
Thank you for participating in Quiz 'O' Cruise-II. <br>\
We hope you enjoyed the event .<br><br>\
Please fill this form to let us know how we can improve :- <a href='https://forms.gle/NhoCd1FKNmgmrrit7'>Link to form</a>")

def main():
    names,names2, emails = get_contacts("QUIZ 'O' CRUISE - II.csv") # read contacts
    message_template = read_template('message.txt')

    # set up the SMTP server
    s = smtplib.SMTP(host='smtp.gmail.com', port=587)
    s.starttls()
    s.login(MY_ADDRESS, PASSWORD)

    i=0;
    # For each contact, send the email:
    for name,name2, email in zip(names,names2, emails):
        msg = MIMEMultipart()       # create a message

        # add in the actual person name to the message template
        message = message_template.substitute(PERSON_NAME=name.title(),PERSON_NAME2=name2,NUMBER=getnumber(i))
        i+=1

        # Prints out the message body for our sake
        print(message)

        # setup the parameters of the message
        msg['From']=MY_ADDRESS
        msg['To']=email
        msg['Subject']="Thank you for registering in Quiz 'O' Cruise"
        
        # add in the message body
        msg.attach(MIMEText(message, 'html'))
        
        filename = "Feedback.jpeg"  # In same directory as script

        # Open PDF file in binary mode
        with open(filename, "rb") as attachment:
            # Add file as application/octet-stream
            # Email client can usually download this automatically as attachment
            part = MIMEBase("application", "octet-stream")
            part.set_payload(attachment.read())

        # Encode file in ASCII characters to send by email    
        encoders.encode_base64(part)

        # Add header as key/value pair to attachment part
        part.add_header(
            "Content-Disposition",
            f"attachment; filename= {filename}",
        )

        # Add attachment to message and convert message to string
        msg.attach(part)

        
        
        # send the message via the server set up earlier.
        s.send_message(msg)
        del msg
        
    # Terminate the SMTP session and close the connection
    s.quit()
    
if __name__ == '__main__':
    main()